In [12]:
import arcpy
import os

#Define the workspace
arcpy.env.workspace = r"Z:\Documents\ArcGIS\Projects\Active Transportation\Active Transportation.gdb"

#Allow features and files to be overwritten 
arcpy.env.overwriteOutput = True

In [13]:
# Define a function to return unique values
def unique_values(table ,field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor if row[0] is not None})

In [14]:
# Create a function to re-map field values
def remap_field_values(fc,input_field,output_field,map_dictionary):
    update_fields = []
    update_fields.append(input_field)
    update_fields.append(output_field)
    with arcpy.da.UpdateCursor(in_table=fc,
                               field_names=update_fields) as cursor:
        for row in cursor:
            # store the Join value of the row being updated in a key variable
            key = row[0]
            # verify that the key is in the Dictionary
            if key in map_dictionary:
                row[1] = map_dictionary[key]
                cursor.updateRow(row)
    del cursor

In [15]:
#define a function for adding standard class fields & source
def add_required_fields(in_table):
    #Create list of lists with field name, data type, length (if applicable), and field alias
    list_of_lists = []
    source = ['source','TEXT','Source',100,None,None]
    existing_class = ['ex_class','SHORT','Existing Class',None,None,None]
    planned_class = ['pln_class','SHORT','Planned Class',None,None,None]
    list_of_lists.extend([source,existing_class,planned_class])
    #Add fields to feature class
    arcpy.management.AddFields(in_table,list_of_lists)

In [67]:
# Create a function to re-map multiple column values to one column
def remap_class_multi_columns(in_table,exist_field,plan_field,exist_map_dict,plan_map_dict):
    update_fields = []
    update_fields.extend([exist_field,plan_field,'ex_class','pln_class'])
    with arcpy.da.UpdateCursor(in_table=in_table,
                           field_names=update_fields) as cursor:
        for row in cursor:
            exist_key = row[0]
            pln_key = row[1]
            if (exist_key != None and exist_key != ' ') and (pln_key != None and pln_key != ' '):
                # store the Join value of the row being updated in a key variable for the existing field
                # verify that the key is in the Dictionary
                row[2] = exist_map_dict[exist_key]
                row[3] = plan_map_dict[pln_key]
                cursor.updateRow(row)
            elif (exist_key != None and exist_key != ' ') and (pln_key == None or pln_key == ' '):
                row[2] = exist_map_dict[exist_key]
                row[3] = None
                cursor.updateRow(row)
            elif (exist_key == None or exist_key == ' ') and (pln_key != None and pln_key != ' '):
                row[2] = None
                row[3] = plan_map_dict[pln_key]
                cursor.updateRow(row)
            else:
                row[2] = None
                row[3] = None
                cursor.updateRow(row)
    del cursor  

In [17]:
# Create a function to re-map class from one column to two columns depending on class value
def remap_class_single_to_multi_columns(in_table,class_field,status_field,class_map_dict,status_map_dict):
    update_fields = []
    update_fields.extend([class_field,status_field,'ex_class','pln_class'])
    with arcpy.da.UpdateCursor(in_table=in_table,
                               field_names=update_fields) as cursor:
        for row in cursor:
            class_key = row[0]
            status_key = row[1]
            if class_key != None and class_key != ' ' and status_map_dict[status_key] == 'Existing':
                row[2] = class_map_dict[class_key]
                cursor.updateRow(row)
            elif class_key != None and class_key != ' ' and status_map_dict[status_key] == 'Planned':
                row[3] = class_map_dict[class_key]
                cursor.updateRow(row)
            else:
                row[2] = None
                row[3] = None
                cursor.updateRow(row)
    del cursor  

In [18]:
# define a function to update the source field
def update_source(in_table,field_name,source):
    # update source column
    with arcpy.da.UpdateCursor(in_table=in_table,field_names=field_name) as source_cursor:
        for row in source_cursor:
            row[0] = source
            source_cursor.updateRow(row)
    del source_cursor

In [19]:
# # define a function to update status where multiple class columns exist and no stand-alone status column exists
# def update_status_multi_class_fields(in_table,exist_field,proposed_field,status_field_name):
#     update_fields = []
#     update_fields.extend([exist_field,proposed_field,status_field_name])
#     with arcpy.da.UpdateCursor(in_table=in_table,
#                                field_names=update_fields) as status_cursor:
#         for row in status_cursor:
#             if row[0] != None and row[0] != ' ':
#                 row[2] = 'Existing'
#                 status_cursor.updateRow(row)
#             elif (row[1] != None and row[1] != ' ') and (row[0] == ' ' or row[0] == None):
#                 row[2] = 'Proposed'
#                 status_cursor.updateRow(row)
#             else:
#                 row[2] = None
#                 status_cursor.updateRow(row)
#     del status_cursor

In [20]:
# Create function to select specific fields to map when using field mapping for a geoprocessing function
def issolate_fields(output_fields, field_mappings):
    for field in field_mappings.fields:
        if field.name not in output_fields:
            field_mappings.removeFieldMap(field_mappings.findFieldMapIndex(field.name))

In [21]:
# Create function to add only selected fields to field mapping object
def create_field_mapping_with_selected_fields(feature_classes,output_fields):
    fm = arcpy.FieldMappings()
    for fc in feature_classes:
        fm.addTable(fc)
        issolate_fields(output_fields=output_fields,field_mappings=fm)
    return fm

In [22]:
def merge_multiple_datasets(dataset_list,select_output_fields,output_fc):
    fm = create_field_mapping_with_selected_fields(feature_classes=dataset_list,
                                               output_fields=select_output_fields)
    print('Selected field list:')
    for field in fm.fields:
        print(field.name)
    print("""\n""")
            
    print("""Merging feature classes...\n""")
    merge_fc = arcpy.Merge_management(inputs=dataset_list,
                                           output=output_fc,
                                           field_mappings=fm,
                                           add_source='NO_SOURCE_INFO')
    print("""Merge complete""")
    return merge_fc

## Update Marin County Attributes

In [11]:
tam_bike_nw = 'tam_bike_network_epsg29610'
add_required_fields(in_table=tam_bike_nw)

In [13]:
# Get unique values for existingbi class field
exist_field = 'ExistingBi'
exist = unique_values(table=tam_bike_nw,field=exist_field)

In [15]:
# Get unique values for proposed class field
pln_field = 'ProposedBi'
planned = unique_values(table=tam_bike_nw,field=pln_field)

In [16]:
# Create dictonary with unique values as keys
dict.fromkeys(exist,'')

{' ': '', 'Class I': '', 'Class II': '', 'Class II / III Hybrid': '', 'Class II / IIIs Hybrid': '', 'Class III': '', 'Class IIIs': '', 'Class IIr': '', 'Class IIr / III Hybrid': '', 'FWY Rte': '', 'Other': ''}

In [17]:
# Create dictonary with unique values as keys
dict.fromkeys(planned,'')

{' ': '', 'Class I': '', 'Class II': '', 'Class III': '', 'Class IIr': '', 'Class IV (two-way)': ''}

In [18]:
# Create a dictionary to map existing facilities class to standard numeric class
exist_dict = {'Class I': 1, 
              'Class II': 2, 
              'Class II / III Hybrid': 2, 
              'Class II / IIIs Hybrid': 2, 
              'Class III': 3, 
              'Class IIIs': 3, 
              'Class IIr': 2, 
              'Class IIr / III Hybrid': 2, 
              'FWY Rte': None, 
              'Other': None}

In [19]:
planned_dict = {'Class I': 1, 
                 'Class II': 2, 
                 'Class III': 3, 
                 'Class IIr': 2, 
                 'Class IV (two-way)': 4}

In [20]:
# remap proposed class values to standard numeric class values
remap_class_multi_columns(in_table=tam_bike_nw,
                          exist_field=exist_field,
                          plan_field=pln_field,
                          exist_map_dict=exist_dict,
                          plan_map_dict=planned_dict)

In [21]:
# update source column
update_source(in_table=tam_bike_nw,
              field_name='source',
              source='Transportation Authority of Marin')

## Update Sonoma County Attributes

In [48]:
scta_bike_nw = 'scta_bike_network_epsg26910'
add_required_fields(in_table=scta_bike_nw)

In [49]:
# get unique class values
class_field = 'ORIG_CLASS'
classes = unique_values(table=scta_bike_nw,field=class_field)

In [50]:
# Get unique status values
status_field = 'GEN_PLAN'
status = unique_values(table=scta_bike_nw,field=status_field)

In [51]:
dict.fromkeys(classes,'')

{' ': '', 'I': '', 'II': '', 'IIB': '', 'III': '', 'III2': '', 'IIIB': '', 'IV': '', 'PED': '', 'PP': '', 'SR': '', 'STDY': '', 'TRL': ''}

In [52]:
dict.fromkeys(status,'')

{' ': '', 'BRIDGE/TUNNEL': '', 'EXISTING': '', 'PROPOSED': ''}

In [53]:
class_dict = {'I': 1, 
              'II': 2, 
              'IIB': 2, 
              'III': 3, 
              'III2': 3, 
              'IIIB': 3, 
              'IV': 4, 
              'PED': None, 
              'PP': None, 
              'SR': None, 
              'STDY': None, 
              'TRL': None}

In [54]:
status_dict = {' ': None, 
              'BRIDGE/TUNNEL': 'Planned', 
              'EXISTING': 'Existing', 
              'PROPOSED': 'Planned'}

In [56]:
remap_class_single_to_multi_columns(in_table=scta_bike_nw,
                                    class_field=class_field,
                                    status_field=status_field,
                                    class_map_dict=class_dict,
                                    status_map_dict=status_dict)

In [57]:
# Update source column
update_source(in_table=scta_bike_nw,
              field_name='source',
              source='Sonoma County Transportation Authority')

## Update Napa County Attributes

In [73]:
nvta_bike_nw = 'nvta_bike_network_epsg26910'
add_required_fields(in_table=nvta_bike_nw)

In [74]:
# get unique class values
class_field = 'fac_type'
classes = unique_values(table=nvta_bike_nw,field=class_field)

In [75]:
# Get unique status values
status_field = 'orig_status'
status = unique_values(table=nvta_bike_nw,field=status_field)

In [76]:
dict.fromkeys(classes,'')

{'Class I Multi Use Path': '', 'Class II Bike Lane': '', 'Class III Bike Boulevard': '', 'Class III Bike Route': '', 'Class IV Separated Bike Lane': '', 'Corridor Study': ''}

In [77]:
dict.fromkeys(status,'')

{'Existing': '', 'Proposed': '', 'Proposed - pending study': ''}

In [78]:
class_dict = {'Class I Multi Use Path': 1, 
              'Class II Bike Lane': 2, 
              'Class III Bike Boulevard': 3, 
              'Class III Bike Route': 3, 
              'Class IV Separated Bike Lane': 4, 
              'Corridor Study': None}

In [79]:
status_dict = {'Existing': 'Existing', 
              'Proposed': 'Planned', 
              'Proposed - pending study': 'Planned'}

In [80]:
remap_class_single_to_multi_columns(in_table=nvta_bike_nw,
                                    class_field=class_field,
                                    status_field=status_field,
                                    class_map_dict=class_dict,
                                    status_map_dict=status_dict)

In [66]:
# Update source column
update_source(in_table=nvta_bike_nw,
              field_name='source',
              source='Napa Valley Transportation Authority')

## Update Solano County Attributes

In [82]:
sta_bike_nw = 'sta_bike_network_epsg26910'
add_required_fields(in_table=sta_bike_nw)

In [83]:
# get unique class values
exist_class = 'existing'
classes = unique_values(table=sta_bike_nw,field=exist_class)

In [85]:
pln_class = 'rec'
planned = unique_values(table=sta_bike_nw,field=pln_class)

In [88]:
dict.fromkeys(classes,'')

{' ': '', 'Class I Multi-Use Path': '', 'Class II Bicycle Lane': '', 'Class II Buffered Bicycle Lane': '', 'Class III Bicycle Boulevard': '', 'Class III Bicycle Route': '', 'Class IV Separated Bikeway': ''}

In [89]:
dict.fromkeys(planned,'')

{' ': '', 'Class I Multi-Use Path': '', 'Class II Bicycle Lane': '', 'Class II Buffered Bicycle Lane': '', 'Class III Bicycle Boulevard': '', 'Class III Bicycle Route': '', 'Class IV Separated Bikeway': '', 'Feasibility Study': ''}

In [92]:
exist_dict = {
              'Class I Multi-Use Path': 1, 
              'Class II Bicycle Lane': 2, 
              'Class II Buffered Bicycle Lane': 2, 
              'Class III Bicycle Boulevard': 3, 
              'Class III Bicycle Route': 3, 
              'Class IV Separated Bikeway': 4}

In [93]:
prop_dict = { 
             'Class I Multi-Use Path': 1, 
             'Class II Bicycle Lane': 2, 
             'Class II Buffered Bicycle Lane': 2, 
             'Class III Bicycle Boulevard': 3, 
             'Class III Bicycle Route': 3, 
             'Class IV Separated Bikeway': 4, 
             'Feasibility Study': None}

In [94]:
remap_class_multi_columns(in_table=sta_bike_nw,
                          exist_field=exist_class,
                          plan_field=pln_class,
                          exist_map_dict=exist_dict,
                          plan_map_dict=prop_dict)

In [96]:
# Update source column
update_source(in_table=sta_bike_nw,
              field_name='source',
              source='Solano Transportation Authority')

## Update Contra Costa Attributes

In [24]:
ccta_bike_nw = 'ccta_bike_network_epsg26910'
add_required_fields(in_table=ccta_bike_nw)

In [25]:
exist_class = 'class'
classes = unique_values(table=ccta_bike_nw,field=exist_class)

In [29]:
status_field = 'status'
status = unique_values(table=ccta_bike_nw,field=status_field)

In [26]:
dict.fromkeys(classes,)

{0: None, 1: None, 2: None, 3: None, 4: None, 5: None}

In [30]:
dict.fromkeys(status,)

{'Existing': None, 'Proposed': None}

In [27]:
# Typically we would not keep the 0 class but this should be kept for Contra Costa. 0 is any bike facility that is unpaved or dirt. 

exist_dict = {0: 0, 
              1: 1, 
              2: 2, 
              3: 3, 
              4: 4, 
              5: 5}

In [31]:
status_dict = {'Existing': 'Existing', 
               'Proposed': 'Planned'}

In [32]:
remap_class_single_to_multi_columns(in_table=ccta_bike_nw,
                                    class_field=exist_class,
                                    status_field=status_field,
                                    class_map_dict=exist_dict,
                                    status_map_dict=status_dict)

In [33]:
update_source(in_table=ccta_bike_nw,
              field_name='source',
              source='Contra Costa Transportation Authority')

## Update Alameda County Attributes

In [34]:
actc_bike_nw = 'actc_bike_network_epsg26910'
add_required_fields(in_table=actc_bike_nw)

In [38]:
# get unique class values
exist_field = 'existing_f'
classes = unique_values(table=actc_bike_nw,field=exist_field)

In [39]:
planned_field = 'planned_fa'
proposed = unique_values(table=actc_bike_nw,field=planned_field)

In [40]:
dict.fromkeys(classes,'')

{' ': '', 'Class I - Shared Use Path': '', 'Class II - Bike Lane': '', 'Class III - Shared Lane': '', 'Class IIIe - Bike Boulevard': '', 'Class IIb - Buffered Bike Lane': '', 'Class IV - Proteced Bike Lane': ''}

In [41]:
dict.fromkeys(proposed,'')

{' ': '', 'Class I - Shared Use Path': '', 'Class II - Bike Lane': '', 'Class III - Shared Lane': '', 'Class IIIe - Bike Boulevard': '', 'Class IIb - Buffered Bike Lane': '', 'Class IV - Proteced Bike Lane': '', 'Study Corridor': ''}

In [44]:
exist_dict = {'Class I - Shared Use Path': 1, 
              'Class II - Bike Lane': 2, 
              'Class III - Shared Lane': 3, 
              'Class IIIe - Bike Boulevard': 3, 
              'Class IIb - Buffered Bike Lane': 2, 
              'Class IV - Proteced Bike Lane': 4}

In [45]:
prop_dict = {'Class I - Shared Use Path': 1, 
             'Class II - Bike Lane': 2, 
             'Class III - Shared Lane': 3, 
             'Class IIIe - Bike Boulevard': 3, 
             'Class IIb - Buffered Bike Lane': 2, 
             'Class IV - Proteced Bike Lane': 4, 
             'Study Corridor': None}

In [46]:
remap_class_multi_columns(in_table=actc_bike_nw,
                          exist_field=exist_field,
                          plan_field=planned_field,
                          exist_map_dict=exist_dict,
                          plan_map_dict=prop_dict)

In [47]:
# Update source column
update_source(in_table=actc_bike_nw,
              field_name='source',
              source='Alameda County Transportation Authority')

## Update Santa Clara County Attributes

In [48]:
vta_bike_nw = 'vta_bike_network_epsg26910'
add_required_fields(in_table=vta_bike_nw)

In [49]:
# get unique class values
class_field = 'BikeClassN'
classes = unique_values(table=vta_bike_nw,field=class_field)

In [51]:
# get unique status values
status_field = 'ExPrStatus'
status = unique_values(table=vta_bike_nw,field=status_field)

In [52]:
dict.fromkeys(classes,'')

{0: '', 1: '', 2: '', 3: '', 4: ''}

In [53]:
dict.fromkeys(status,'')

{'Existing': '', 'Proposed': ''}

In [54]:
exist_dict = {0: None, 
              1: 1, 
              2: 2, 
              3: 3, 
              4: 4}

In [55]:
status_dict = {'Existing': 'Existing', 
               'Proposed': 'Planned'}

In [56]:
remap_class_single_to_multi_columns(in_table=vta_bike_nw,
                                    class_field=class_field,
                                    status_field=status_field,
                                    class_map_dict=exist_dict,
                                    status_map_dict=status_dict)

In [57]:
update_source(in_table=vta_bike_nw,
              field_name='source',
              source='Valley Transportation Authority')

## Update San Mateo County Attributes

In [59]:
ccag_bike_nw = 'ccag_bike_network_epsg26910'
add_required_fields(in_table=ccag_bike_nw)

In [60]:
exist_field = 'existing_facility'
exist = unique_values(table=ccag_bike_nw,
                      field=exist_field)

In [61]:
prop_field = 'final_recommended_facility'
proposed = unique_values(table=ccag_bike_nw,
                        field=prop_field)

In [62]:
dict.fromkeys(exist,'')

{'Class 1 Path': '', 'Class 2 Bicycle Lane': '', 'Class 2b Buffered Bicycle Lane': '', 'Class 3 Bicycle Route': '', 'Class 3b Bicycle Boulevard': '', 'Class 4 Separated Bicycle Lane': ''}

In [63]:
dict.fromkeys(proposed,'')

{'Class 1 Path': '', 'Class 2 Bicycle Lane': '', 'Class 2b Buffered Bicycle Lane': '', 'Class 3 Bicycle Route': '', 'Class 3b Bicycle Boulevard': '', 'Class 3c Bicycle Route with Wide Shoulders': '', 'Class 4 Separated Bicycle Lane': '', 'Keep Exising Facility': '', 'Undetermined Facility Type': ''}

In [64]:
exist_dict = {'Class 1 Path': 1, 
              'Class 2 Bicycle Lane': 2, 
              'Class 2b Buffered Bicycle Lane': 2, 
              'Class 3 Bicycle Route': 3, 
              'Class 3b Bicycle Boulevard': 3, 
              'Class 4 Separated Bicycle Lane': 4}

In [65]:
prop_dict = {'Class 1 Path': 1, 
             'Class 2 Bicycle Lane': 2, 
             'Class 2b Buffered Bicycle Lane': 2, 
             'Class 3 Bicycle Route': 3, 
             'Class 3b Bicycle Boulevard': 3, 
             'Class 3c Bicycle Route with Wide Shoulders': 3, 
             'Class 4 Separated Bicycle Lane': 4, 
             'Keep Exising Facility': None, 
             'Undetermined Facility Type': None}

In [68]:
remap_class_multi_columns(in_table=ccag_bike_nw,
                          exist_field=exist_field,
                          plan_field=prop_field,
                          exist_map_dict=exist_dict,
                          plan_map_dict=prop_dict)

In [69]:
update_source(in_table=ccag_bike_nw,
              field_name='source',
              source='City/County Association of Governments San Mateo')

## Update San Francisco County Attributes

In [70]:
sfcta_bike_nw = 'sfcta_bike_network_epsg26910'
add_required_fields(in_table=sfcta_bike_nw)

In [71]:
#get unique class values
class_field = 'FACILITY_T'
classes = unique_values(table=sfcta_bike_nw,field=class_field)

In [72]:
dict.fromkeys(classes,'')

{'CLASS I': '', 'CLASS II': '', 'CLASS III': '', 'CLASS IV': '', 'CLASS iii': ''}

In [73]:
exist_dict = {'CLASS I': 1, 
              'CLASS II': 2, 
              'CLASS III': 3, 
              'CLASS IV': 4, 
              'CLASS iii': 3}

In [75]:
remap_field_values(fc=sfcta_bike_nw,
                   input_field=class_field,
                   output_field='ex_class',
                   map_dictionary=exist_dict)

In [76]:
#Update source column 

update_source(in_table=sfcta_bike_nw,field_name='source',source='San Francisco County Transportation Authority')

## Merge County Bike Networks

In [87]:
arcpy.ListFeatureClasses()

['regional_bikeway_network_2019', 'nvta_bike_network_epsg26910', 'sta_bike_network_epsg26910', 'sfcta_bike_network_epsg26910', 'actc_bike_network_epsg26910', 'vta_bike_network_epsg26910', 'ccta_bike_network_epsg26910', 'tam_bike_network_epsg29610', 'ccag_bike_network_epsg26910', 'scta_bike_network_epsg26910', 'regional_bike_facilities_2021_epsg26910', 'regional_bikeway_network_2019_epsg26910', 'oakland_bike_network_epsg26910']

In [89]:
dataset_list = ['nvta_bike_network_epsg26910', 
                'sta_bike_network_epsg26910', 
                'sfcta_bike_network_epsg26910', 
                'actc_bike_network_epsg26910', 
                'vta_bike_network_epsg26910', 
                'ccta_bike_network_epsg26910', 
                'tam_bike_network_epsg29610', 
                'ccag_bike_network_epsg26910', 
                'scta_bike_network_epsg26910']
output_fields = ['source','ex_class','pln_class']
regional_bike_facilities = 'regional_bike_facilities_2021_epsg26910'
merge_multiple_datasets(dataset_list=dataset_list,
                        select_output_fields=output_fields,
                        output_fc=regional_bike_facilities)

Selected field list:
source
ex_class
pln_class


Merging feature classes...

Merge complete


<Result 'Z:\\Documents\\ArcGIS\\Projects\\Active Transportation\\Active Transportation.gdb\\regional_bike_facilities_2021_epsg26910'>

## Update Oakland Attributes

In [78]:
oak_bike_nw = 'oakland_bike_network_epsg26910'
add_required_fields(in_table=oak_bike_nw)

In [79]:
exist_field = 'existingcl'
exist = unique_values(table=oak_bike_nw,field=exist_field)

In [80]:
prop_field = 'proposedcl'
proposed = unique_values(table=oak_bike_nw,field=prop_field)

In [81]:
dict.fromkeys(exist,'')

{'0': '', '0.0': '', '1': '', '1X': '', '2': '', '2.0': '', '2.3': '', '2.3A': '', '2.3B': '', '2B': '', '2B.0': '', '2B.2': '', '2B.3A': '', '3': '', '3.0': '', '3A': '', '3A.0': '', '3A.3': '', '3B': '', '4': '', '4.2': '', '4.2B': '', '4.3A': ''}

In [82]:
dict.fromkeys(proposed,'')

{'0': '', '1': '', '2': '', '2.0': '', '2.3A': '', '2.3B': '', '2B': '', '2B.0': '', '2B.2': '', '2B.3A': '', '3': '', '3A': '', '3B': '', '4': ''}

In [83]:
exist_dict = {'0': None, 
              '0.0': None, 
              '1': 1, 
              '1X': 1, 
              '2': 2, 
              '2.0': 2, 
              '2.3': 2, 
              '2.3A': 2, 
              '2.3B': 2, 
              '2B': 2, 
              '2B.0': 2, 
              '2B.2': 2, 
              '2B.3A': 2, 
              '3': 3, 
              '3.0': 3, 
              '3A': 3, 
              '3A.0': 3, 
              '3A.3': 3, 
              '3B': 3, 
              '4': 4, 
              '4.2': 4, 
              '4.2B': 4, 
              '4.3A': 4}

In [84]:
prop_dict = {'0': None, 
             '1': 1, 
             '2': 2, 
             '2.0': 2, 
             '2.3A': 2, 
             '2.3B': 2, 
             '2B': 2, 
             '2B.0': 2, 
             '2B.2': 2, 
             '2B.3A': 2, 
             '3': 3, 
             '3A': 3, 
             '3B': 3, 
             '4': 4}

In [85]:
remap_class_multi_columns(in_table=oak_bike_nw,
                          exist_field=exist_field,
                          plan_field=prop_field,
                          exist_map_dict=exist_dict,
                          plan_map_dict=prop_dict)

In [21]:
# update_fields = ['existingcl','proposedcl','status']
# with arcpy.da.UpdateCursor(in_table=oak_bike_nw,
#                            field_names=update_fields) as status_cursor:
#     for row in status_cursor:
#         if row[0] != '0' and row[0] != '0.0':
#             row[2] = 'Existing'
#             status_cursor.updateRow(row)
#         elif (row[1] != 0) and (row[0] == '0' or row[0] == '0.0'):
#             row[2] = 'Proposed'
#             status_cursor.updateRow(row)
#         else:
#             row[2] = None
#             status_cursor.updateRow(row)
# del status_cursor

In [86]:
update_source(in_table=oak_bike_nw,field_name='source',source='City of Oakland')

## Update Bay Area Trails Collaborative Attributes

In [90]:
batc_bike_nw = 'batc_bike_network_epsg26910'
add_required_fields(in_table=batc_bike_nw)

In [91]:
exist_field = 'TrailClass'
classes = unique_values(table=batc_bike_nw,field=exist_field)

In [92]:
dict.fromkeys(classes,'')

{'Class 1 bikeway': '', 'Class 2 bikeway': '', 'Class 3 bikeway': '', 'Class 4 bikeway': '', 'Multi-use trail': '', 'Pedestrian-only trail': '', 'TBD': '', 'Uncertain': ''}

In [93]:
class_dict = {'Class 1 bikeway': 1, 
              'Class 2 bikeway': 2, 
              'Class 3 bikeway': 3, 
              'Class 4 bikeway': 4, 
              'Multi-use trail': 1, 
              'Pedestrian-only trail': None, 
              'TBD': None, 
              'Uncertain': None}

In [95]:
remap_field_values(fc=batc_bike_nw,
                   input_field='TrailClass',
                   output_field='ex_class',
                   map_dictionary=class_dict)

In [96]:
update_source(in_table=batc_bike_nw,field_name='source',source='Bay Area Trails Collaborative')